In [1]:
from load_data import * 
from analyze_data import * 
import sys
import warnings
import matplotlib.pyplot as plt
from SWRmodule import *
sys.path.append('/home1/efeghhi/ripple_memory/')
from brain_labels import HPC_labels, ENT_labels, PHC_labels, temporal_lobe_labels,\
                        MFG_labels, IFG_labels, nonHPC_MTL_labels, ENTPHC_labels, AMY_labels
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
import statsmodels.formula.api as smf

/home1/efeghhi/.conda/envs/env3/lib/python3.7/site-packages/ptsa/data/timeseries.py:526: FutureWarning: xarray subclass TimeSeriesX should explicitly define __slots__
  class TimeSeriesX(TimeSeries):
/home1/efeghhi/.conda/envs/env3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
encoding_mode = 1

catFR_dir = '/scratch/efeghhi/catFR1/ENCODING/'

region_name = '' # if empty string, loads all data

# load all data
data_dict = load_data(catFR_dir, region_name=region_name, encoding_mode=encoding_mode, 
                      condition_on_ca1_ripples=True)

if encoding_mode: 
    data_dict = remove_wrong_length_lists(data_dict)
    
# ca1
ca1_elecs = [x for x in HPC_labels if 'ca1' in x]
data_dict_ca1 = select_region(data_dict, ca1_elecs)
count_num_trials(data_dict_ca1, "ca1")


Number of trials in ca1: 113028


In [89]:
dd_trials['HFA'].shape

(113028, 150)

In [90]:
detectRipplesHamming(dd_trials['HFA'], 5, 50, np.zeros_like(dd_trials['HFA']))

ValueError: Invalid cutoff frequency: frequencies must be greater than 0 and less than fs/2.

In [42]:
def create_ripple_exists(dd_trials, ripple_start, ripple_end, min_ripple_time):
    
    ripple_exists_idxs = np.argwhere(np.sum(dd_trials['ripple'][:, ripple_start:ripple_end],axis=1)>min_ripple_time)
    ripple_exists = np.zeros(dd_trials['ripple'].shape[0])
    ripple_exists[ripple_exists_idxs] = 1
    
    return ripple_exists

In [66]:
clustered_int = create_semantic_clustered_array(data_dict_ca1, encoding_mode)
data_dict_ca1['clust_int'] = clustered_int

In [67]:
dd_trials = dict_to_numpy(data_dict_ca1, order='C')

order: C


In [68]:
dd_trials.keys()

dict_keys(['ripple', 'HFA', 'theta', 'theta_phase', 'clust', 'correct', 'position', 'list_num', 'subj', 'sess', 'elec_names', 'serial_pos', 'clust_int'])

In [69]:
ripple_exists = create_ripple_exists(dd_trials, 550, 900, 0)
dd_trials['ripple_exists'] = ripple_exists

In [70]:
dd_trials_pd = pd.DataFrame({'HFA_mean': np.mean(dd_trials['HFA'][:, 55:90],axis=1), 
                             'clust': dd_trials['clust_int'],
                             'ripple_exists':ripple_exists, 
                             'correct':dd_trials['correct'], 
                             'subj':dd_trials['subj'], 
                             'sess':dd_trials['sess']})
vc = {'sess':'0+sess'}

In [76]:
print(dd_trials_pd.shape)
dd_trials_pd_no_ripple = dd_trials_pd.loc[dd_trials_pd.ripple_exists==0]
print(dd_trials_pd_no_ripple.shape)

(113028, 6)
(90335, 6)


In [102]:
get_bin_CI_model = smf.mixedlm("correct ~ 1 + HFA_mean", dd_trials_pd, groups="subj", vc_formula=vc, 
                                re_formula='1 + HFA_mean')
bin_model = get_bin_CI_model.fit(reml=True, method='nm',maxiter=2000)
bin_model.summary()

/home1/efeghhi/.conda/envs/env3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
=============================================================
Model:              MixedLM  Dependent Variable:  correct    
No. Observations:   113028   Method:              REML       
No. Groups:         100      Scale:               0.2065     
Min. group size:    84       Likelihood:          -71611.0423
Max. group size:    10500    Converged:           Yes        
Mean group size:    1130.3                                   
-------------------------------------------------------------
                    Coef. Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept           0.317    0.014 22.955 0.000  0.290  0.344
HFA_mean            0.007    0.002  4.376 0.000  0.004  0.010
subj Var            0.015    0.006                           
subj x HFA_mean Cov 0.000    0.000                           
HFA_mean Var        0.000    0.000                           
sess Var            0.005    0.002                           
=============================================================

"""

In [101]:
get_bin_CI_model = smf.mixedlm("correct ~ 1 + HFA_mean", dd_trials_pd_no_ripple, groups="subj", vc_formula=vc, 
                                re_formula='1 + HFA_mean')
bin_model = get_bin_CI_model.fit(reml=True, method='nm',maxiter=2000)
bin_model.summary()

/home1/efeghhi/.conda/envs/env3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
=============================================================
Model:              MixedLM  Dependent Variable:  correct    
No. Observations:   90335    Method:              REML       
No. Groups:         100      Scale:               0.2054     
Min. group size:    78       Likelihood:          -57037.4438
Max. group size:    8155     Converged:           Yes        
Mean group size:    903.4                                    
-------------------------------------------------------------
                    Coef. Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept           0.318    0.014 22.935 0.000  0.291  0.346
HFA_mean            0.010    0.002  5.695 0.000  0.006  0.013
subj Var            0.016    0.006                           
subj x HFA_mean Cov 0.001    0.001                           
HFA_mean Var        0.000    0.000                           
sess Var            0.005    0.002                           
=============================================================

"""

In [81]:
dd_trials_pd_clust = dd_trials_pd.loc[dd_trials_pd.clust != -1]
print(dd_trials_pd_clust.shape)
dd_trials_pd_clust_no_ripple = dd_trials_pd_clust.loc[dd_trials_pd_clust.ripple_exists==0]
print(dd_trials_pd_clust_no_ripple.shape)
dd_trials_pd_clust_ripple = dd_trials_pd_clust.loc[dd_trials_pd_clust.ripple_exists==1]

(35829, 6)
(28123, 6)


In [86]:
dd_trials_pd_clust_ripple.loc[dd_trials_pd_clust_ripple.clust==0]['HFA_mean'].mean() - dd_trials_pd_clust_no_ripple.loc[dd_trials_pd_clust_no_ripple.clust==0]['HFA_mean'].mean()

1.4887903614780365

In [87]:
dd_trials_pd_clust_ripple.loc[dd_trials_pd_clust_ripple.clust==1]['HFA_mean'].mean() - dd_trials_pd_clust_no_ripple.loc[dd_trials_pd_clust_no_ripple.clust==1]['HFA_mean'].mean()

1.4908461407256102

In [100]:
get_bin_CI_model = smf.mixedlm("clust ~ 1 + HFA_mean", dd_trials_pd_clust, groups="subj", vc_formula=vc, 
                                re_formula='1 + HFA_mean')
bin_model = get_bin_CI_model.fit(reml=True, method='nm',maxiter=2000)
bin_model.summary()

/home1/efeghhi/.conda/envs/env3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
=============================================================
Model:              MixedLM  Dependent Variable:  clust      
No. Observations:   35829    Method:              REML       
No. Groups:         100      Scale:               0.2387     
Min. group size:    8        Likelihood:          -25353.8008
Max. group size:    6613     Converged:           Yes        
Mean group size:    358.3                                    
-------------------------------------------------------------
                    Coef. Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept           0.489    0.011 44.662 0.000  0.468  0.511
HFA_mean            0.008    0.002  3.440 0.001  0.004  0.013
subj Var            0.008    0.004                           
subj x HFA_mean Cov 0.000    0.001                           
HFA_mean Var        0.000    0.000                           
sess Var            0.003    0.001                           
=============================================================

"""

In [99]:
get_bin_CI_model = smf.mixedlm("clust ~ 1 + HFA_mean", dd_trials_pd_clust_no_ripple, groups="subj", vc_formula=vc, 
                                re_formula='1 + HFA_mean')
bin_model = get_bin_CI_model.fit(reml=True, method='nm',maxiter=2000)
bin_model.summary()

/home1/efeghhi/.conda/envs/env3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
=============================================================
Model:              MixedLM  Dependent Variable:  clust      
No. Observations:   28123    Method:              REML       
No. Groups:         100      Scale:               0.2399     
Min. group size:    7        Likelihood:          -19985.5499
Max. group size:    5070     Converged:           Yes        
Mean group size:    281.2                                    
-------------------------------------------------------------
                    Coef. Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept           0.491    0.011 44.326 0.000  0.469  0.512
HFA_mean            0.009    0.003  3.600 0.000  0.004  0.014
subj Var            0.008    0.004                           
subj x HFA_mean Cov 0.000    0.001                           
HFA_mean Var        0.000    0.000                           
sess Var            0.003    0.002                           
=============================================================

"""

In [96]:
get_bin_CI_model = smf.mixedlm("clust ~ 1 + ripple_exists", dd_trials_pd_clust, groups="subj", vc_formula=vc, 
                                re_formula='1 + ripple_exists')
bin_model = get_bin_CI_model.fit(reml=True, method='nm',maxiter=2000)
bin_model.summary()

/home1/efeghhi/.conda/envs/env3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home1/efeghhi/.conda/envs/env3/lib/python3.7/site-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


<class 'statsmodels.iolib.summary2.Summary'>
"""
              Mixed Linear Model Regression Results
==================================================================
Model:               MixedLM    Dependent Variable:    clust      
No. Observations:    35829      Method:                REML       
No. Groups:          100        Scale:                 0.2393     
Min. group size:     8          Likelihood:            -25381.1485
Max. group size:     6613       Converged:             Yes        
Mean group size:     358.3                                        
------------------------------------------------------------------
                         Coef. Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept                0.486    0.011 43.908 0.000  0.464  0.508
ripple_exists            0.030    0.009  3.433 0.001  0.013  0.047
subj Var                 0.008    0.004                           
subj x ripple_exists Cov 0.000    0.001                           
ripple_exists Var        0.000                                    
sess Var                 0.003    0.001                           
==================================================================

"""

In [97]:
vc = {'sess':'0+sess'}
get_bin_CI_model = smf.mixedlm("correct ~ 1 + ripple_exists", dd_trials_pd, groups="subj", vc_formula=vc, 
                                re_formula='1 + ripple_exists')
bin_model = get_bin_CI_model.fit(reml=True, method='nm',maxiter=2000)
bin_model.summary()

/home1/efeghhi/.conda/envs/env3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
              Mixed Linear Model Regression Results
==================================================================
Model:               MixedLM    Dependent Variable:    correct    
No. Observations:    113028     Method:                REML       
No. Groups:          100        Scale:                 0.2073     
Min. group size:     84         Likelihood:            -71798.4990
Max. group size:     10500      Converged:             Yes        
Mean group size:     1130.3                                       
------------------------------------------------------------------
                         Coef. Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept                0.317    0.014 22.864 0.000  0.290  0.344
ripple_exists            0.005    0.005  0.907 0.364 -0.005  0.015
subj Var                 0.016    0.006                           
subj x ripple_exists Cov 0.000    0.001                           
ripple_exists Var        0.001    0.001                           
sess Var                 0.005    0.002                           
==================================================================

"""